# Issue 46: add functionality to choose minimizing or maximizing for each metric in a dataset

[See issue progress on GitHub here.](https://github.com/NREL/tyche/issues/46)

In [1]:
import os
import sys

sys.path.insert(1, os.path.abspath("../../src"))

import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

# Sustainable Aviation Fuel

In [2]:
saf_loc = "../../tutorial/data/saf"

saf_designs = ty.Designs(saf_loc)
saf_investments = ty.Investments(saf_loc)
saf_designs.compile()
tranche_results = saf_investments.evaluate_tranches(saf_designs,
                                                    sample_count=1000)

techs = tranche_results.summary.index.get_level_values('Technology').unique()

Evaluating HEFA Camelina
Evaluating HEFA Castor
Evaluating HEFA Jatropha
Evaluating HEFA Pennycress
Evaluating HEFA Yellow Grease


Optimize for minimum jet selling price (MJSP) by minimizing MJSP directly.

In [3]:
min_results = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator,
        sense = 'min',
    )
    metric_max = optimizer.max_metrics()
    w = optimizer.opt_slsqp(
        "MJSP",
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    min_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3],
        'Maximum metrics': metric_max,
        'Optimization sense' : w[5]
    }
    min_results.append(
        min_dict
    )
    del tranche_tech_results, evaluator, optimizer, metric_max, min_dict, w

min_results

HEFA Camelina - Optimization terminated successfully
HEFA Castor - Optimization terminated successfully
HEFA Jatropha - Optimization terminated successfully
HEFA Pennycress - Optimization terminated successfully
HEFA Yellow Grease - Iteration limit reached


[{'Technology': 'HEFA Camelina',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            0.000000e+00
  Fatty Acid Composition        2.600000e+06
  Overall Process Efficiency    2.400000e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.090892
  MJSP                    26.583685
  Reduction in Jet GHG     0.050108
  Reduction in MJSP        2.432156
  Name: Value, dtype: float64,
  'Maximum metrics': Jet GHG                 6.292125e-02
  MJSP                    2.425130e+01
  Reduction in Jet GHG    4.413090e-18
  Reduction in MJSP      -9.197589e-07
  Name: Value, dtype: float64,
  'Optimization sense': 'min'},
 {'Technology': 'HEFA Castor',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            1.750016e+06
  Fatty Acid Composition        1.749999e+06
  Overall Process Efficiency    1.499985e+06
  Na

Now optimize for MJSP by maximizing the reduction in MSJP from the state of technology value.

In [5]:
max_results = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator,
        sense = 'max',
    )
    metric_max = optimizer.max_metrics()
    w = optimizer.opt_slsqp(
        "Reduction in MJSP",
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    max_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3],
        'Maximum metrics': metric_max,
        'Optimization sense' : w[5]
    }
    max_results.append(
        max_dict
    )
    del tranche_tech_results, evaluator, optimizer, metric_max, max_dict, w

max_results

HEFA Camelina - Optimization terminated successfully
HEFA Castor - Optimization terminated successfully
HEFA Jatropha - Optimization terminated successfully
HEFA Pennycress - Optimization terminated successfully
HEFA Yellow Grease - Optimization terminated successfully


[{'Technology': 'HEFA Camelina',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            4.912108e-04
  Fatty Acid Composition        2.600000e+06
  Overall Process Efficiency    2.400000e+06
  Name: Amount, dtype: float64,
  'Optimum metrics': Index
  Jet GHG                  0.090892
  MJSP                    26.583685
  Reduction in Jet GHG     0.050108
  Reduction in MJSP        2.432156
  Name: Value, dtype: float64,
  'Maximum metrics': Jet GHG                  0.141000
  MJSP                    29.015842
  Reduction in Jet GHG     0.078079
  Reduction in MJSP        4.764545
  Name: Value, dtype: float64,
  'Optimization sense': 'max'},
 {'Technology': 'HEFA Castor',
  'Optimizer status': 'Optimization terminated successfully',
  'Investment amounts': Category
  Co-Product Revenue            1.750016e+06
  Fatty Acid Composition        1.749999e+06
  Overall Process Efficiency    1.499985e+06
  Name: Amount, 

The metric values found from `optimizer.max_metrics()` consistently out-perform the metric values found through `opt_slsqp`.